In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
import PyPDF2
from PyPDF2 import PdfFileReader
import re
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
import sys
from sklearn import linear_model
warnings.filterwarnings("ignore")
sys.path.append(r'C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\asc-library-temp')
from PIL import Image
from pytesseract import pytesseract
from pdf2image import convert_from_path
from flatten_json import flatten_json
import json
import torch
import codecs

In [3]:
from library import Dataset
from finbert_senti import get_finbert_sentiments

In [4]:
driver = webdriver.Chrome(r'C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\chromedriver.exe')

In [24]:
last_date = 'Apr 12 2022'
type_lst = ['Company Comment - Bull Bear', 'Company Comment', 'Company Brief', 'Initiation of Coverage - RJA']
headline = []
analyst = []
broker_label = []
url = []
flag = False
for i in range(10):
    for j in range(1,11):
        if driver.find_element_by_xpath('//*[@id="allReports"]/tbody/tr[{}]/td[4]'.format(j)).text != last_date:
            if driver.find_element_by_xpath('//*[@id="allReports"]/tbody/tr[{}]/td[3]'.format(j)).text in type_lst:
                headline.append(driver.find_element_by_xpath('//*[@id="allReports"]/tbody/tr[{}]/td[1]/a[1]'.format(j)).text)
                analyst.append(driver.find_element_by_xpath('//*[@id="allReports"]/tbody/tr[{}]/td[2]'.format(j)).text)
                broker_label.append(driver.find_element_by_xpath('//*[@id="allReports"]/tbody/tr[{}]/td[3]'.format(j)).text)
                url.append(driver.find_element_by_xpath('//*[@id="allReports"]/tbody/tr[{}]/td[1]/a[1]'.format(j)).get_attribute('href'))
        else:
            flag = True
            break
    if flag == True:
        break    
    driver.find_element_by_xpath('//*[@id="allReports_next"]/a').click()
df = pd.DataFrame({'headline' : headline,
                  'analyst_pri' : analyst,
                  'report_type_broker' : broker_label,
                  'url' : url}, 
                    columns=['headline','analyst_pri', 'report_type_broker','url'])

In [88]:
for i in range(len(df)):
    try:
        driver.get(df.at[i,'url'].replace('pdf','html'))
        df.at[i,'ticker'] = driver.find_element_by_xpath('//*[@id="xsltTemplate"]/table/tbody/tr[1]/td/table/tbody/tr[2]/td[1]/table/tbody/tr[2]/td').text.split('\n')[0]
        df.at[i,'sector'] = driver.find_element_by_xpath('//*[@id="xsltTemplate"]/table/tbody/tr[1]/td/table/tbody/tr[2]/td[1]/table/tbody/tr[2]/td').text.split('\n')[1]
        df.at[i,'summary'] = driver.find_element_by_xpath('//*[@id="webedit"]/table/tbody/tr/td/table/tbody/tr/td/p[1]').text
        df.at[i,'useful'] = driver.find_element_by_xpath('//*[@id="xsltTemplate"]/table/tbody/tr[1]/td/table/tbody/tr[2]/td[3]/table/tbody/tr[4]/td/table[1]/tbody/tr[2]/td').text
        df.at[i,'publish_date_and_time'] = driver.find_element_by_xpath('//*[@id="xsltTemplate"]/table/tbody/tr[1]/td/table/tbody/tr[2]/td[3]/table/tbody/tr[2]/td').text.replace('|', ' ')
    except:
        pass

In [89]:
a_dict = {'NASDAQ ' : 'US',
          'NYSE ' : 'US',
         'TSX ' : 'CN',
         'TSXV ': 'CN',
         'NEO ': 'CN',
         'MADRID ': 'SM',
         'NASDAQ' : 'US',
          'NYSE' : 'US',
         'TSX' : 'CN',
         'TSXV': 'CN',
         'NEO': 'CN',
         'MADRID': 'SM',}
for i in range(len(df)):
    try:
        df.at[i,'ticker'] = df.at[i,'ticker'].split('(')[1]
        df.at[i,'ticker'] = df.at[i,'ticker'].split(')')[0]
        df.at[i,'ticker'] = df.at[i,'ticker'].split('|')[0]
        df.at[i,'ticker'] = df.at[i,'ticker'].split('-')[0] + ' ' + a_dict[df.at[i,'ticker'].split('-')[1]]
    except:
        pass

In [91]:
df = df.dropna(subset=['useful']).reset_index(drop=True)
for i in range(len(df)):
    try:
        df.at[i,'uid'] = df.at[i,'url'].split('encrypt=')[1]
        df.at[i,'uid'] = df.at[i,'uid'].split('&')[0]
        df.at[i,'publish_date_and_time'] = df.at[i,'publish_date_and_time'].replace(' EDT','')
        df.at[i,'rating_curr'] = df.at[i,'useful'].split('\n')[0]
        df.at[i,'tp_curr'] = df.at[i,'useful'].split('\n')[1]
    except:
        pass
    
for i in range(len(df)):
    try:
        if len(df.at[i,'rating_curr'].split('old:'))==1:
            df.at[i,'rating_prev'] = df.at[i,'rating_curr']
        else:
            df.at[i,'rating_prev'] = df.at[i,'rating_curr'].split('old:')[1]
            df.at[i,'rating_curr'] = df.at[i,'rating_curr'].split('old:')[0]
            df.at[i,'rating_curr'] = df.at[i,'rating_curr'].replace(' ↑ ','')
            df.at[i,'rating_curr'] = df.at[i,'rating_curr'].replace(' ↓ ','')
    except:
        pass
    
for i in range(len(df)):
    try:
        if len(df.at[i,'tp_curr'].split('old:'))==1:
            df.at[i,'tp_prev'] = df.at[i,'tp_curr'].replace('Target Price ','')
            df.at[i,'tp_curr'] = df.at[i,'tp_curr'].replace('Target Price ','')
            df.at[i,'tp_prev'] = df.at[i,'tp_prev'].replace('NM','')
            df.at[i,'tp_curr'] = df.at[i,'tp_curr'].replace('NM','')
        else:
            df.at[i,'tp_prev'] = df.at[i,'tp_curr'].split('old:')[1]
            df.at[i,'tp_curr'] = df.at[i,'tp_curr'].split('old:')[0].replace('Target Price ','')
            df.at[i,'tp_curr'] = df.at[i,'tp_curr'].replace(' ↑ ','')
            df.at[i,'tp_curr'] = df.at[i,'tp_curr'].replace(' ↓ ','')
            df.at[i,'tp_prev'] = df.at[i,'tp_prev'].replace('NM','')
            df.at[i,'tp_curr'] = df.at[i,'tp_curr'].replace('NM','')
    except:
        pass
del df['useful']
df = df.fillna('')

In [92]:
data = Dataset(df)
data.clean(tz_name='America/New_York')
df = data.df

Tickers ['ERF US Equity' 'FORA CN Equity' 'OGI CN Equity' 'OSK CN Equity'] are not in the trading days json file.
Tickers [['DAL US Equity', '2022-04-13'], ['FAST US Equity', '2022-04-13'], ['GDRX US Equity', '2022-04-13'], ['CMBM US Equity', '2022-04-13'], ['FRC US Equity', '2022-04-13'], ['XBC CN Equity', '2022-04-13'], ['PRQR US Equity', '2022-04-13'], ['OPS CN Equity', '2022-04-13'], ['SIL CN Equity', '2022-04-13'], ['FRC US Equity', '2022-04-13'], ['CMCSA US Equity', '2022-04-13'], ['FAST US Equity', '2022-04-13'], ['RGLD US Equity', '2022-04-13'], ['OBE CN Equity', '2022-04-13'], ['CBSH US Equity', '2022-04-13']] are in the trading days json file, but cannot return an entry date.


In [101]:
ear = pd.read_excel(r"C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\ern_announcement_datetime_data.xlsx", index_col = 0)

df['date_local'] = pd.to_datetime(df['date_local'])
ear['release_date'] = pd.to_datetime(ear['release_date'])

for i in df.index:
    ticker = df.loc[i,'ticker']
    temp = ear[ear['ticker'] == ticker]
    date = df.loc[i,'date_local']
    
    try:  df.loc[i,'last_date'] = min(temp[temp['release_date'] <= date]['release_date'], key=lambda x: abs(pd.to_datetime(x) - pd.to_datetime(date)))
    except:  df.loc[i,'last_date'] = np.NaN
    try: df.loc[i,'next_date'] = min(temp[temp['release_date'] > date]['release_date'], key=lambda x: abs(pd.to_datetime(x) - pd.to_datetime(date)))
    except: df.loc[i,'next_date'] = np.NaN
    try:  df.loc[i, 'closest_date'] = min(temp['release_date'], key=lambda x: abs(pd.to_datetime(x) - pd.to_datetime(date)))
    except: df.loc[i,'closest_date'] = np.NaN       
df['date_local'] = pd.to_datetime(df['date_local'])
df['next_date'] = pd.to_datetime(df['next_date'])
df['last_date'] = pd.to_datetime(df['last_date'])
df['closest_date'] = pd.to_datetime(df['closest_date'])

In [102]:
class ern_label_classi_():
    def __init__(self, df):
        self.df = df
        self.keyword_list = ['q1', 'q2', 'q3', 'q4', 'h1', 'h2', 'fy', 'profit alert', 'profit warning']
        self.keywords = "|".join(self.keyword_list)    
        self.preliminary = ['preliminary']
        self.preliminary = "|".join(self.preliminary)  
    def relabel_earnings(self):
        ##Keyword Filter
        self.df['label'] = np.where( (self.df['summary'].apply(lambda x: x.lower()).str.contains(self.keywords,regex=True)) | (self.df['headline'].apply(lambda x: x.lower()).str.contains(self.keywords,regex=True)), 'QR', 'False')
        ##Specify date time
        self.df['label'] = np.where(self.df['closest_date'] == self.df['date_local'], 'QRT0', self.df['label'])
        self.df['label'] = np.where( (self.df['date_local'] - self.df['last_date']).dt.days == 1, 'QRT1', self.df['label'])
        self.df['label'] = np.where((self.df['date_local'] - self.df['last_date']).dt.days == 2, 'QRT2', self.df['label'])
        self.df['label'] = np.where((self.df['date_local'] - self.df['next_date']).dt.days == -1, 'QRT-1', self.df['label'])
        self.df['label'] = np.where( ((self.df['summary'].apply(lambda x: x.lower()).str.contains(self.preliminary, regex=True)) | (self.df['headline'].apply(lambda x: x.lower()).str.contains(self.preliminary, regex=True)) ), 'QRP', self.df['label'])
        self.df['label'] = np.where( (self.df['summary'].apply(lambda x: x.lower()).str.contains(' guidance| outlook')) | (self.df['headline'].apply(lambda x: x.lower()).str.contains(' guidance| outlook')) , 'G', self.df['label'])
    def clean_prelim(self):
        self.confusion = ['will publish', 'will report', 'already publish', 'already announced', 'already prereleased', 'already released']
        self.confusion = "|".join(self.confusion)  
        temp = self.df[self.df['label'] == 'QRP']
        temp['label'] = np.where( (temp['summary'].apply(lambda x: x.lower()).str.contains(self.confusion,regex=True)) | (temp['headline'].apply(lambda x: x.lower()).str.contains(self.confusion,regex=True)), 'G', temp['label'])
        self.df.loc[temp.index, 'label'] = temp['label']

In [103]:
ern_label_classi = ern_label_classi_(df)
ern_label_classi.relabel_earnings()
ern_label_classi.clean_prelim()

In [104]:
df

,headline,analyst_pri,report_type_broker,url,ticker,sector,summary,rating_curr,rating_prev,tp_curr,...,exch_region,date_local,time_local,release_period,ticker_updated,entry_date,last_date,next_date,closest_date,label
0,DAL | Anything You Can Do I Can Do Better,Savanthi Syth,Company Comment - Bull Bear,http://raymondjames.bluematrix.com/sellside/Em...,DAL US,Airlines | Network Airlines,Delta’s characteristic optimism gained credenc...,Strong Buy 1,Strong Buy 1,$53.00,...,Americas,2022-04-13,22:30:00,After,DAL US,NaT,NaT,NaT,NaT,G
1,FAST | Unrealistic Implied L/T Growth Overshad...,Sam J. Darkatsh,Company Comment,http://raymondjames.bluematrix.com/sellside/Em...,FAST US,Specialty/Industrial Distribution,Summary: Reiterate Underperform. Fastenal’s 1Q...,Underperform 4,Underperform 4,,...,Americas,2022-04-13,22:12:00,After,FAST US,NaT,NaT,NaT,NaT,QR
2,GDRX | Reversal of the 2021 Trends in Short In...,John W. Ransom,Company Brief,http://raymondjames.bluematrix.com/sellside/Em...,GDRX US,Medical Technology,"In this note, we find that two trends that wer...",Outperform 2,Outperform 2,,...,Americas,2022-04-13,20:09:00,After,GDRX US,NaT,NaT,NaT,NaT,False
3,"CMBM | 1Q22 Negative Pre-Announcement, CFO Rep...",Simon Leopold,Company Comment,http://raymondjames.bluematrix.com/sellside/Em...,CMBM US,Data Infrastructure,"We maintain our Strong Buy rating on Cambium, ...",Strong Buy 1,Strong Buy 1,$43.00,...,Americas,2022-04-13,20:03:00,After,CMBM US,NaT,NaT,NaT,NaT,G
4,FORA | Tapping Commercial Value Of Enthusiast ...,Steven Li,Company Comment,http://raymondjames.bluematrix.com/sellside/Em...,FORA CN,Software/IT Services,"Enthusiast communities (forums, apps, or marke...",Outperform 2,Outperform 2,C$32.00,...,Americas,2022-04-13,16:00:00,Within,FORA CN,NaT,NaT,NaT,NaT,False
5,FRC | Raising EPS Estimates; Maintaining Marke...,David J. Long,Company Comment,http://raymondjames.bluematrix.com/sellside/Em...,FRC US,Banking,We maintain our Market Perform rating on FRC s...,Market Perform 3,Market Perform 3,,...,Americas,2022-04-13,15:49:00,Within,FRC US,NaT,NaT,NaT,NaT,False
6,XBC | Announces Large Order in Carbon Capture ...,David Quezada,Company Comment,http://raymondjames.bluematrix.com/sellside/Em...,XBC CN,Power & Energy Infrastructure | Diversified & ...,As we highlighted at the time of our recent up...,Outperform 2,Outperform 2,C$3.50,...,Americas,2022-04-13,15:35:00,Within,XBC CN,NaT,NaT,NaT,NaT,False
7,PRQR | Updated Strategy: Prioritizing Ultevurs...,Steven Seedhouse,Company Brief,http://raymondjames.bluematrix.com/sellside/Em...,PRQR US,Biotechnology,We remain Market Perform on PRQR following tod...,Market Perform 3,Market Perform 3,,...,Americas,2022-04-13,14:28:00,Within,PRQR US,NaT,NaT,NaT,NaT,QR
8,OPS | Reit. Strong Buy: TAVR Catalysts Begin J...,Rahul Sarugaser,Company Brief,http://raymondjames.bluematrix.com/sellside/Em...,OPS CN,Biotechnology,"Today, Opsens (OPS-TSX, SB1) reported its 2Q22...",Strong Buy 1,Strong Buy 1,,...,Americas,2022-04-13,11:59:00,Within,OPS CN,NaT,NaT,NaT,NaT,QR
9,SIL | Potential 25 mln Ag eq oz (and growing) ...,Craig Stanley,Company Brief,http://raymondjames.bluematrix.com/sellside/Em...,SIL CN,Precious Metals - Silver,SilverCrest announced drill assays from its El...,Outperform 2,Outperform 2,,...,Americas,2022-04-13,11:34:00,Within,SIL CN,NaT,NaT,NaT,NaT,False
